![http://cognitiveatlas.org//static/img/logo-front.png](http://cognitiveatlas.org//static/img/logo-front.png)


<hr>

Here we are going to interact with the [Cognitive Atlas](https://www.cognitiveatlas.org) 
via it's [Python API](https://github.com/CognitiveAtlas/cogat-python).

## What is the Cognitive Atlas?

The Cognitive Atlas is a community developed knowledge base for concepts defined in cognitive psychology. More formally, this is called an ontology, and less formally, it's a place to look up what a term means that you see in the psychology literature, and how it relates to other terms.

We start using the cognitive atlas when we have a question of interest.

> What is working memory?

That seems like a good question! How might we have investigated the question, without any additional tool?

 - Do a Google Search and trust the internet
 - Do a literature review and try to synthesize many results
 - Rely on meta analysis
 
The above options are good, but each has drawbacks. For example, it would be hard to synthesize the opinions and studies from any of the resources above. Meta analyses are good, 

<hr>

## The Web Interface

If you are just exploring, you are best to interact with the Cognitive Atlas via it's web interface at [cognitiveatlas.org](https://www.cognitiveatlas.org). For example, here is a page for working memory at [http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/). I'm showing you the URL for a few important reasons:

  - we see that it's something called a "concept"
  - we see embedded in the URL is some kind of "id," the string `trm_4a3fd79d0b5a7`. Let's keep this in mind for later.

### Use Cases

From the interface we can quickly learn many things like the following:

### 1. Definition

We might be writing a review or paper, or just want to understand what the term means, to the best of the knowledge of the experts that have contributed to the Cognitive Atlas. Here we can see, front and center, this definition (here is the first bit of it).

> "active maintenance and flexible updating of goal/task relevant information (items, goals, strategies, etc.) in a form that resists interference but has limited capacity. ..."

This is great, but note that it hasn't been reviewed. We can do more investigation by looking at the [citations](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-biblio) on the page, or in the case of a non Anonymous contribution, see the contribution author(s). Note that although you can't see it, there is a complete history kept of contributors to the terms.

### 2. Tasks

If you are a graduate student and wanting to conduct a study to measure working memory, this might be your stopping point! There is a list of over 15 [tasks](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-measurements) on the page that have been asserted to measure working memory.

### 3. Relationships

How does working memory relate to the concept of [maintenance](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0ba30/)? What about general [decision making](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0a038/)? Remember that the Cognitive Atlas is an ontology, which means it defines both concepts **and** relationships between them! We have descriptors for these relationships, like "part of" or "kind of". You might see how this maps nicely to a graph:

 - a **concept** or **task** (or general entity represented in the atlas) maps to a node
 - a **relationship** between terms maps to a link between nodes
 - the strength of any relationship maps to a weight of a link between nodes
 
 This opens up a very powerful idea - we can use ideas and methods from graph theory to understand ideas in cognitive science. We can also generate visualizations to see them. For example, try posting this query:
 
 ```cypher
 MATCH (n:concept) where n.id = 'trm_4a3fd79d0ba30' OPTIONAL MATCH (n)-[]-(r) return n, r
 ```
Into the Neo4J browser (the top line of the window at [https://cognitiveatlas.org:7473/browser/](https://cognitiveatlas.org:7473/browser/)). You would see a little graph that describes the term "maintenance"

![neo4j-graph-maintenance.png](neo4j-graph-maintenance.png)

While many of these interactive, jumpy visualizations aren't super useful, they can be helpful for a talk, to show an idea in a paper, or solidify your understanding of a more complex relationship.

These are the basic functions and components of the Cognitive Atlas. If you are interested, you can also explore [Phenotypes](http://cognitiveatlas.org/disorders), [Tasks](http://cognitiveatlas.org/tasks/a/) and [Collections](http://cognitiveatlas.org/collections) (groups of tasks). But the web interface is limited in what we are able to analyze, in a large scale sense. In the next section, let's talk about more programmatic use of the Cognitive Atlas.

##

In [1]:
import cognitiveatlas